![](img/330-banner.png)

# Lecture 16: Introduction to natural language processing 

UBC 2022-23

Instructor: Varada Kolhatkar

In [3]:
## Imports

import os
import re
import string
import sys
import time
from collections import Counter, defaultdict

sys.path.append("code/.")

import IPython
import numpy as np
import numpy.random as npr
import pandas as pd
from comat import CooccurrenceMatrix
from IPython.display import HTML
from ipywidgets import interactive
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from preprocessing import MyPreprocessor
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kvarada/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

<br><br>

### Announcements

- Final exam: December 15th at 7:00pm [PHRM 1101](https://learningspaces.ubc.ca/classrooms/phrm-1101)
- Homework 7 will be released on Tuesday will be due in a week.  
- There will be 2 more homework assignments after this. The schedule is posted on the website.  

## Learning objectives

- Broadly explain what is natural language processing (NLP). 
- Name some common NLP applications. 
- Explain the general idea of a vector space model.
- Explain the difference between different word representations: term-term co-occurrence matrix representation and Word2Vec representation.
- Describe the reasons and benefits of using pre-trained embeddings. 
- Load and use pre-trained word embeddings to find word similarities and analogies. 
- Demonstrate biases in embeddings and learn to watch out for such biases in pre-trained embeddings.
- Use word embeddings in text classification and document clustering using `spaCy`.
- Explain the general idea of topic modeling. 
- Describe the input and output of topic modeling. 
- Carry out basic text preprocessing using `spaCy`.   

<br><br>

## What is Natural Language Processing (NLP)?

- What should a search engine return when asked the following question? 

![](img/lexical_ambiguity.png)

<!-- <center> -->
<!-- <img src="img/lexical_ambiguity.png" width="1000" height="1000"> -->
<!-- </center> -->

### What is Natural Language Processing (NLP)?
#### How often do you search everyday? 

![](img/Google_search.png)

<!-- <center> -->
<!-- <img src="img/Google_search.png" width="900" height="900"> -->
<!-- </center> -->

### What is Natural Language Processing (NLP)?

![](img/WhatisNLP.png)

<!-- <center> -->
<!-- <img src="img/WhatisNLP.png" width="800" height="800"> -->
<!-- </center> -->    

### Everyday NLP applications

![](img/annotation-image.png)
<!-- <center> -->
<!-- <img src="img/annotation-image.png" height="1200" width="1200"> -->
<!-- </center> -->

### NLP in news 

Often you'll NLP in news. Some examples: 
- [How suicide prevention is getting a boost from artificial intelligence](https://abcnews.go.com/GMA/Wellness/suicide-prevention-boost-artificial-intelligence-exclusive/story?id=76541481)
- [Meet GPT-3. It Has Learned to Code (and Blog and Argue).](https://www.nytimes.com/2020/11/24/science/artificial-intelligence-ai-gpt3.html)
- [How Do You Know a Human Wrote This?](https://www.nytimes.com/2020/07/29/opinion/gpt-3-ai-automation.html)
- ...

### Why is NLP hard?

- Language is complex and subtle. 
- Language is ambiguous at different levels. 
- Language understanding involves common-sense knowledge and real-world reasoning.
- All the problems related to representation and reasoning in artificial intelligence arise in this domain. 

### Example: Lexical ambiguity

<br><br>

![](img/lexical_ambiguity.png)

<!-- <img src="img/lexical_ambiguity.png" width="1000" height="1000"> -->
    

### Example: Referential ambiguity
<br><br>

<!-- <img src="img/referential_ambiguity.png" width="1000" height="1000"> -->

![](img/referential_ambiguity.png)
    

### [Ambiguous news headlines](http://www.fun-with-words.com/ambiguous_headlines.html)

<blockquote>
PROSTITUTES APPEAL TO POPE
</blockquote>    

- **appeal to** means make a serious or urgent request or be attractive or interesting?

<blockquote>
KICKING BABY CONSIDERED TO BE HEALTHY    
</blockquote> 

- **kicking** is used as an adjective or a verb?

<blockquote>
MILK DRINKERS ARE TURNING TO POWDER
</blockquote>

- **turning** means becoming or take up?

### Overall goal

- Give you a quick introduction to you of this important field in artificial intelligence which extensively used machine learning.   

![](img/NLP_in_industry.png)

<!-- <center> -->
<!-- <img src="img/NLP_in_industry.png" width="900" height="800"> -->
<!-- </center> -->

Today's plan

- Word embeddings
- Topic modeling 
- Basic text preprocessing 

<br><br>

## Word embeddings 

- How can we represent word meaning? 
- The idea is to create **representations of words** so that distances between words in the feature space indicate relationships between them. 

![](img/t-SNE_word_embeddings.png)
<!-- <center>
<img src="img/t-SNE_word_embeddings.png" width="850" height="850">
</center>
 -->
 
(Attribution: [Jurafsky and Martin 3rd edition](https://web.stanford.edu/~jurafsky/slp3/))

### Why do we care about word representations?

- So far we have been talking about sentence or document representations. 
- Now we are going one step back and talking about word representations. 
- Word is a basic unit of text and in order to capture meaning of text it is useful to capture word meaning (e.g., in terms of relationships between words).  

### Word meaning 

- A favourite topic of philosophers for centuries. 
- An example from legal domain: [Are hockey gloves "gloves, mittens, mitts" or "articles of plastics"?](https://www.scc-csc.ca/case-dossier/info/sum-som-eng.aspx?cas=36258)

<blockquote>
Canada (A.G.) v. Igloo Vikski Inc. was a tariff code case that made its way to the SCC (Supreme Court of Canada). The case disputed the definition of hockey gloves as either "gloves, mittens, or mitts" or as "other articles of plastic."
</blockquote>

![](img/hockey_gloves_case.png)
<!-- <center>
<img src="img/hockey_gloves_case.png" width="500" height="500">
</center>
 -->

### Word meaning: ML and Natural Language Processing (NLP) view
- Modeling word meaning that allows us to 
    - draw useful inferences to solve meaning-related problems 
    - find relationship between words, e.g., which words are similar, which ones have positive or negative connotations
    

### Word similarity   

- Suppose you are carrying out sentiment analysis. 
- Consider the sentences below. 

> S1: **The movie was awesome!!** 

> S2: **The movie was amazing!!** 

> S3: **The movie was awful!!** 


- Here we would like to capture similarity between **awesome** and **amazing** in reference to sentiment analysis task. 

<br><br>

### Word representations

### Activity 1:  Brainstorm ways to represent words (~2 mins) 

- Suppose you are building a question answering system and you are given the following question and three candidate answers. 
- What kind of relationship between words would we like our representation to capture in order to arrive at the correct answer?  
    
<blockquote>       
<p style="font-size:30px"><b>Question:</b> How <b>tall</b> is Machu Picchu?</p>
    <p style="font-size:30px"><b>Candidate 1:</b> Machu Picchu is 13.164 degrees south of the equator.</p>    
<p style="font-size:30px"><b>Candidate 2:</b> The official height of Machu Picchu is 2,430 m.</p>
<p style="font-size:30px"><b>Candidate 3:</b> Machu Picchu is 80 kilometres (50 miles) northwest of Cusco.</p>    
</blockquote> 
    

### Need a representation that captures relationships between words.

- We will be looking at two such representations.  
    1. Sparse representation with **term-term co-occurrence matrix**
    2. Dense representation with **word2vec skip-gram model**

### Term-term co-occurrence matrix

- So far we have been talking about documents and we created document-term co-occurrence matrix (e.g., bag-of-words representation of text). 
- We can also do this with words. The idea is to go through a corpus of text, keeping a count of all of the words that appear in context of each word (within a window).

- An example: 

<!-- <center>
<img src="img/term-term_comat.png" width="600" height="600">
</center>
     -->
(Credit: Jurafsky and Martin 3rd edition)


### Visualizing word vectors and similarity 

![](img/word_vectors_and_angles.png)
<!-- <center>
<img src="img/word_vectors_and_angles.png" width="800" height="800">
</center>
     -->
(Credit: Jurafsky and Martin 3rd edition)

- The similarity is calculated using dot products between word vectors.
    - Example: $\vec{\text{digital}}.\vec{\text{information}} = 0 \times 1 + 1\times 6 = 6$
    - Higher the dot product more similar the words.

### Visualizing word vectors and similarity

<img src="img/word_vectors_and_angles.png" width="600" height="600">

(Credit: Jurafsky and Martin 3rd edition)

- The similarity is calculated using dot products between word vectors.
    - Example: $\vec{\text{digital}}.\vec{\text{information}} = 0 \times 1 + 1\times 6 = 6$
    - Higher the dot product more similar the words.

- We can also calculate a normalized version of dot products. 
    $$similarity_{cosine}(w_1,w_2) = \frac{w_1.w_2}{\left\lVert w_1\right\rVert_2 \left\lVert w_2\right\rVert_2}$$


In [5]:
corpus = [
    "How tall is Machu Picchu?",
    "Machu Picchu is 13.164 degrees south of the equator.",
    "The official height of Machu Picchu is 2,430 m.",
    "Machu Picchu is 80 kilometres (50 miles) northwest of Cusco.",
    "It is 80 kilometres (50 miles) northwest of Cusco, on the crest of the mountain Machu Picchu, located about 2,430 metres (7,970 feet) above mean sea level, over 1,000 metres (3,300 ft) lower than Cusco, which has an elevation of 3,400 metres (11,200 ft).",
]
sents = MyPreprocessor(corpus)

In [6]:
cm = CooccurrenceMatrix(
    sents, window_size=2
)  # Let's build term-term co-occurrence matrix for our text.
comat = cm.fit_transform()
vocab = cm.get_feature_names()
df = pd.DataFrame(comat.todense(), columns=vocab, index=vocab, dtype=np.int8)
df.head()

,tall,machu,picchu,13.164,degrees,south,equator,official,height,"2,430",...,mean,sea,level,"1,000","3,300",ft,lower,elevation,"3,400","11,200"
tall,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
machu,1,0,5,1,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
picchu,1,5,0,1,1,0,0,0,1,2,...,0,0,0,0,0,0,0,0,0,0
13.164,0,1,1,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
degrees,0,0,1,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
from sklearn.metrics.pairwise import cosine_similarity


def print_cosine_similarity(df, word1, word2):
    """
    Returns similarity score between word1 and word2
    Arguments
    ---------
    df    -- (pandas.DataFrame)
        Dataframe containing word representations
    word1 -- (array)
        Representation of word1
    word2 -- (array)
        Representation of word2

    Returns
    --------
    None. Returns similarity score between word1 and word2 with the given representation
    """
    vec1 = df.loc[word1].to_numpy().reshape(1, -1)
    vec2 = df.loc[word2].to_numpy().reshape(1, -1)
    sim = cosine_similarity(vec1, vec2)
    print(
        "The dot product between %s and %s: %0.2f and cosine similarity is: %0.2f"
        % (word1, word2, np.dot(vec1.flatten(), vec2.flatten()), sim)
    )

In [8]:
print_cosine_similarity(df, "tall", "height")
print_cosine_similarity(df, "tall", "official")

The dot product between tall and height: 2.00 and cosine similarity is: 0.82
The dot product between tall and official: 1.00 and cosine similarity is: 0.50


- We are getting non-zero cosine similarity now and we are able to capture some similarities between words now. 
- That said similarities do not make much sense in the toy example above because we're using a tiny corpus. 
- To find meaningful patterns of similarities between words, we need a large corpus. 
- Let's try a bit larger corpus and check whether the similarities make sense. 

In [9]:
import wikipedia
from nltk.tokenize import sent_tokenize, word_tokenize

corpus = []

queries = [
    "Machu Picchu",
    "Sagarmatha",
    "Sequoia sempervirens",
    "President (country)",
    "Politics Canada",
]

for i in range(len(queries)):
    sents = sent_tokenize(wikipedia.page(queries[i]).content)
    corpus.extend(sents)
print("Number of sentences in the corpus: ", len(corpus))

Number of sentences in the corpus:  1470


In [10]:
sents = MyPreprocessor(corpus)
cm = CooccurrenceMatrix(sents)
comat = cm.fit_transform()
vocab = cm.get_feature_names()
df = pd.DataFrame(comat.todense(), columns=vocab, index=vocab, dtype=np.int8)
df

,machu,picchu,15th-century,inca,citadel,located,eastern,cordillera,southern,peru,...,canadian-politics.com,2008-08-28,comprehensive,cbc,digital,archives,boondoggles,elephants,campaigning,textbook
machu,0,91,1,6,2,2,1,0,1,5,...,0,0,0,0,0,0,0,0,0,0
picchu,91,0,1,7,4,1,1,0,1,4,...,0,0,0,0,0,0,0,0,0,0
15th-century,1,1,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
inca,6,7,1,0,1,1,1,0,0,2,...,0,0,0,0,0,0,0,0,0,0
citadel,2,4,1,1,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
archives,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,2,0,1,1,1,0
boondoggles,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,1,0,0
elephants,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,1,1,0,0,0
campaigning,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0


In [11]:
print_cosine_similarity(df, "tall", "height")
print_cosine_similarity(df, "tall", "official")

The dot product between tall and height: 35.00 and cosine similarity is: 0.28
The dot product between tall and official: 1.00 and cosine similarity is: 0.02


<br><br><br><br>

## Dense representations 

- The goal is to learn general purpose embeddings that are useful for common tasks involving text data. 

### Sparse vs. dense word vectors
- Term-term co-occurrence matrix representation is long and sparse. 
    - length |V| is usually large (e.g., > 50,000) 
    - most elements are zero
- OK because there are efficient ways to deal with sparse matrices.

### Alternative 

- Learn short (~100 to 1000 dimensions) and dense vectors.  
- Short vectors are usually easier to train with ML models (less weights to train).
- They may generalize better.
- In practice they work much better! 

### Dense word representations

- Before looking at how to create dense word representations let's see how they look like and 
what can we do with them. 
- Below I am loading word vectors trained on Google News corpus using a package called `gensim`. 

```conda install -c anaconda gensim```

In [13]:
import gensim
import gensim.downloader as api

print(list(api.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [14]:
google_news_vectors = api.load('word2vec-google-news-300')

[--------------------------------------------------] 1.4% 23.5/1662.8MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[=-------------------------------------------------] 3.7% 62.3/1662.8MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[===-----------------------------------------------] 6.7% 111.4/1662.8MB downloaded

...

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[================================================--] 96.8% 1610.1/1662.8MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[=================================================-] 99.8% 1659.0/1662.8MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [15]:
print("Size of vocabulary: ", len(google_news_vectors))

Size of vocabulary:  3000000


- `google_news_vectors` above has 300 dimensional word vectors for 3,000,000 unique words from Google news. 

### What can we do with these word vectors?

- Let's examine word vector for the word UBC. 

In [16]:
google_news_vectors["UBC"][:20]  # Representation of the word UBC

array([-0.3828125 , -0.18066406,  0.10644531,  0.4296875 ,  0.21582031,
       -0.10693359,  0.13476562, -0.08740234, -0.14648438, -0.09619141,
        0.02807617,  0.01409912, -0.12890625, -0.21972656, -0.41210938,
       -0.1875    , -0.11914062, -0.22851562,  0.19433594, -0.08642578],
      dtype=float32)

In [18]:
google_news_vectors["UBC"].shape

(300,)

Indeed it is a short and a dense vector! 

### Finding similar words 

- Given word $w$, search in the vector space for the word closest to $w$ as measured by cosine distance. 

In [19]:
google_news_vectors.most_similar("UBC")

[('UVic', 0.788647472858429),
 ('SFU', 0.7588527798652649),
 ('Simon_Fraser', 0.7356575131416321),
 ('UFV', 0.688043475151062),
 ('VIU', 0.6778583526611328),
 ('Kwantlen', 0.6771429181098938),
 ('UBCO', 0.6734487414360046),
 ('UPEI', 0.6731126308441162),
 ('UBC_Okanagan', 0.6709133982658386),
 ('Lakehead_University', 0.6622507572174072)]

In [20]:
google_news_vectors.most_similar("information")

[('info', 0.7363681793212891),
 ('infomation', 0.6800296306610107),
 ('infor_mation', 0.6733849048614502),
 ('informaiton', 0.6639009118080139),
 ('informa_tion', 0.6601256728172302),
 ('informationon', 0.633933424949646),
 ('informationabout', 0.6320979595184326),
 ('Information', 0.6186580061912537),
 ('informaion', 0.6093292236328125),
 ('details', 0.6063088774681091)]

If you want to extract all documents containing words similar to **information**, you could use this information.  

Google News embeddings also support multi-word phrases. 

In [21]:
google_news_vectors.most_similar("british_columbia")

[('alberta', 0.6111123561859131),
 ('canadian', 0.6086405515670776),
 ('ontario', 0.6031432151794434),
 ('erik', 0.5993570685386658),
 ('dominican_republic', 0.5925410985946655),
 ('costco', 0.5824530720710754),
 ('rhode_island', 0.5804311633110046),
 ('dreampharmaceuticals', 0.5755444169044495),
 ('canada', 0.5630921721458435),
 ('austin', 0.5623062252998352)]

### Finding similarity scores between words

In [22]:
google_news_vectors.similarity("Canada", "hockey")

0.27610135

In [23]:
google_news_vectors.similarity("Japan", "hockey")

0.0019627889

In [24]:
word_pairs = [
    ("height", "tall"),
    ("height", "official"),
    ("pineapple", "mango"),
    ("pineapple", "juice"),
    ("sun", "robot"),
    ("gpu", "hummus"),
]
for pair in word_pairs:
    print(
        "The similarity between %s and %s is %0.3f"
        % (pair[0], pair[1], google_news_vectors.similarity(pair[0], pair[1]))
    )

The similarity between height and tall is 0.473
The similarity between height and official is 0.002
The similarity between pineapple and mango is 0.668
The similarity between pineapple and juice is 0.418
The similarity between sun and robot is 0.029
The similarity between gpu and hummus is 0.019


We are getting reasonable word similarity scores!! 

### Short and dense word embeddings 

![](img/word2vec.png)
<!-- <center>
<img src="img/word2vec.png" width="600" height="600">
</center>     -->

### Pre-trained embeddings

- Training embeddings is computationally expensive
- For typical corpora, the vocabulary size is greater than 100,000.  
- If the size of embeddings is 300, the number of parameters of the model is $2 \times 30,000,000$. 
- So people have trained embeddings on huge corpora and made them available.  

A number of pre-trained word embeddings are available. The most popular ones are:  

- [word2vec](https://code.google.com/archive/p/word2vec/)
    * trained on several corpora using the word2vec algorithm 
- [wikipedia2vec](https://wikipedia2vec.github.io/wikipedia2vec/pretrained/)
    * pretrained embeddings for 12 languages 
- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using [the fastText algorithm](http://aclweb.org/anthology/Q17-1010)
    * published by Facebook    

Let's try Google News pre-trained embeddings.  

- You can download pre-trained embeddings from their original source. 
- `Gensim` provides an api to conveniently load them. 

### Success of word2vec

- This analogy example often comes up when people talk about word2vec, which was used by the authors of this method. 
- **MAN : KING :: WOMAN : ?**
    - What is the word that is similar to **WOMAN** in the same sense as **KING** is similar to **MAN**?
- Perform a simple algebraic operations with the vector representation of words.
    $\vec{X} = \vec{\text{KING}} − \vec{\text{MAN}} + \vec{\text{WOMAN}}$
- Search in the vector space for the word closest to $\vec{X}$ measured by cosine distance.

![](img/word_analogies1.png)

<!-- <center>
<img src="img/word_analogies1.png" width="400" height="400">
</center>
     -->
(Credit: Mikolov et al. 2013)  

In [25]:
def analogy(word1, word2, word3, model=google_news_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

In [26]:
analogy("man", "king", "woman")

man : king :: woman : ?


,Analogy word,Score
0,queen,0.711819
1,monarch,0.618967
2,princess,0.590243
3,crown_prince,0.549946
4,prince,0.537732
5,kings,0.523684
6,Queen_Consort,0.523595
7,queens,0.518113
8,sultan,0.509859
9,monarchy,0.508741


In [27]:
analogy("Montreal", "canadiens", "Vancouver")

Montreal : canadiens :: Vancouver : ?


,Analogy word,Score
0,WINTER_GAMES,0.529153
1,canucks,0.525315
2,BRITISH_COLUMBIA,0.524376
3,Chilliwack,0.522794
4,vancouver,0.520225
5,KELOWNA,0.516232
6,Mukmuk,0.508372
7,Lethbridge_Hurricanes,0.503294
8,Kimberly_Joines,0.500331
9,CHILLIWACK,0.498324


In [28]:
analogy("Italy", "pizza", "Japan")

Italy : pizza :: Japan : ?


,Analogy word,Score
0,sushi,0.542754
1,ramen,0.541916
2,bento,0.539965
3,teriyaki,0.502705
4,yakisoba,0.502351
5,takoyaki,0.501550
6,onigiri,0.501540
7,burger,0.495270
8,ramen_noodle,0.493126
9,noodle,0.490753


In [29]:
analogy("USA", "pizza", "India") # Just for fun

USA : pizza :: India : ?


,Analogy word,Score
0,vada_pav,0.554463
1,jalebi,0.547090
2,idlis,0.540039
3,pav_bhaji,0.526046
4,dosas,0.521772
5,samosa,0.520700
6,idli,0.516858
7,pizzas,0.516199
8,tiffin,0.514347
9,chaat,0.508534


So you can imagine these models being useful in many meaning-related tasks. 

![](img/word2vec-country-capitals.png)

(Credit: Mikolov et al. 2013)  

### Examples of semantic and syntactic relationships


<img src="img/word_analogies2.png" width="800" height="800">

(Credit: Mikolov 2013)

### Implicit biases and stereotypes in word embeddings

In [ ]:
analogy("man", "computer_programmer", "woman")

man : computer_programmer :: woman : ?


,Analogy word,Score
0,homemaker,0.562712
1,housewife,0.510505
2,graphic_designer,0.505180
3,schoolteacher,0.497949
4,businesswoman,0.493489
5,paralegal,0.492551
6,registered_nurse,0.490797
7,saleswoman,0.488163
8,electrical_engineer,0.479773
9,mechanical_engineer,0.475540


![](img/eva-srsly.png)

- Embeddings reflect gender stereotypes present in broader society.
- They may also amplify these stereotypes because of their widespread usage. 
- See the paper [Man is to Computer Programmer as Woman is to ...](http://papers.nips.cc/paper/6228-man-is-to-computer-programmer-as-woman-is-to-homemaker-debiasing-word-embeddings.pdf).

Most of the modern embeddings are de-biased for some obvious biases. 

```{note}
Note that these pre-trained word vectors are of big size (in several gigabytes). 
```

<br><br>

### Word vectors with spaCy

- spaCy also gives you access to word vectors with bigger models: `en_core_web_md` or `en_core_web_lr`
- `spaCy`'s pre-trained embeddings are trained on [OntoNotes corpus](https://catalog.ldc.upenn.edu/LDC2013T19).
- This corpus has a collection of different styles of texts such as telephone conversations, newswire, newsgroups, broadcast news, broadcast conversation, weblogs, religious texts. 
- Let's try it out. 

In [ ]:
import spacy

nlp = spacy.load("en_core_web_md")

doc = nlp("pineapple")
doc.vector

<br><br>

### (Optional) Representing documents using word embeddings

- Assuming that we have reasonable representations of words. 
- How do we represent meaning of paragraphs or documents?
- Two simple approaches
    - Averaging embeddings
    - Concatenating embeddings

### Averaging embeddings

<blockquote>
All empty promises
</blockquote>
    
$(embedding(all) + embedding(empty) + embedding(promise))/3$

### Average embeddings with spaCy

- We can do this conveniently with [spaCy](https://spacy.io/usage/linguistic-features#vectors-similarity). 
- We need `en_core_web_md` model to access word vectors. 
- You can download the model by going to command line and in your course `conda` environment and download `en_core_web_md` as follows.   

```
conda activate cpsc330
python -m spacy download en_core_web_md
```

We can access word vectors for individual words in `spaCy` as follows. 

In [ ]:
nlp("empty").vector[0:10]

We can get average embeddings for a sentence or a document in `spaCy` as follows: 

In [ ]:
s = "All empty promises"
doc = nlp(s)
avg_sent_emb = doc.vector
print(avg_sent_emb.shape)
print("Vector for: {}\n{}".format((s), (avg_sent_emb[0:10])))

### Similarity between documents 

- We can also get similarity between documents as follows. 
- Note that this is based on average embeddings of each sentence. 

In [ ]:
doc1 = nlp("Deep learning is very popular these days.")
doc2 = nlp("Machine learning is dominated by neural networks.")
doc3 = nlp("A home-made fresh bread with butter and cheese.")

# Similarity of two documents
print(doc1, "<->", doc2, doc1.similarity(doc2))
print(doc2, "<->", doc3, doc2.similarity(doc3))

- Do these scores make sense? 
- There are no common words, but we are still able to identify that doc1 and doc2 are more similar that doc2 and doc3.  
- You can use such average embedding representation in text classification tasks.

###  Airline sentiment analysis using average embedding representation 

- Let's try average embedding representation for airline sentiment analysis.
- We used this dataset last week so you should already have it in the data directory. If not you can download it [here](https://www.kaggle.com/jaskarancr/airline-sentiment-dataset). 

In [ ]:
df = pd.read_csv("data/Airline-Sentiment-2-w-AA.csv", encoding="ISO-8859-1")

In [ ]:
from sklearn.model_selection import cross_validate, train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=123)
X_train, y_train = train_df["text"], train_df["airline_sentiment"]
X_test, y_test = test_df["text"], test_df["airline_sentiment"]

In [ ]:
train_df.head()

### Bag-of-words representation for sentiment analysis

In [ ]:
pipe = make_pipeline(
    CountVectorizer(stop_words="english"), LogisticRegression(max_iter=1000)
)
pipe.named_steps["countvectorizer"].fit(X_train)
X_train_transformed = pipe.named_steps["countvectorizer"].transform(X_train)
print("Data matrix shape:", X_train_transformed.shape)
pipe.fit(X_train, y_train);

In [ ]:
print("Train accuracy {:.2f}".format(pipe.score(X_train, y_train)))
print("Test accuracy {:.2f}".format(pipe.score(X_test, y_test)))

### Sentiment analysis with average embedding representation

- Let's see how can we get word vectors using `spaCy`. 
- Let's create average embedding representation for each example. 

In [ ]:
X_train_embeddings = pd.DataFrame([text.vector for text in nlp.pipe(X_train)])
X_test_embeddings = pd.DataFrame([text.vector for text in nlp.pipe(X_test)])

We have reduced dimensionality from 13,064 to 300! 

In [ ]:
X_train_embeddings.shape

In [ ]:
X_train_embeddings.head()

### Sentiment classification using average embeddings 

- What are the train and test accuracies with average word embedding representation? 
- The accuracy is a bit better with less overfitting. 
- Note that we are using **transfer learning** here.
- The embeddings are trained on a completely different corpus. 

In [ ]:
lgr = LogisticRegression(max_iter=1000)
lgr.fit(X_train_embeddings, y_train)
print("Train accuracy {:.2f}".format(lgr.score(X_train_embeddings, y_train)))
print("Test accuracy {:.2f}".format(lgr.score(X_test_embeddings, y_test)))

### Sentiment classification using advanced sentence representations 

- Since, representing documents is so essential for text classification tasks, there are more advanced methods for document representation. 
- In homework 7, you also explore sentence embedding representation. 

In [ ]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer("paraphrase-distilroberta-base-v1")

In [ ]:
emb_sents = embedder.encode("all empty promises")
emb_sents.shape

In [ ]:
emb_train = embedder.encode(train_df["text"].tolist())
emb_train_df = pd.DataFrame(emb_train, index=train_df.index)
emb_train_df

In [ ]:
emb_test = embedder.encode(test_df["text"].tolist())
emb_test_df = pd.DataFrame(emb_test, index=test_df.index)
emb_test_df

In [ ]:
lgr = LogisticRegression(max_iter=1000)
lgr.fit(emb_train, y_train)
print("Train accuracy {:.2f}".format(lgr.score(emb_train, y_train)))
print("Test accuracy {:.2f}".format(lgr.score(emb_test, y_test)))

- Some improvement over bag of words and average embedding representations! 
- But much slower ...

## Break (5 min)

![](img/eva-coffee.png)

<br><br><br><br>

## Topic modeling 

### Topic modeling motivation

- Suppose you have a large collection of documents on a variety of topics. 

### Example: A corpus of news articles 

![](img/TM_NYT_articles.png)

<!-- <center> -->
<!-- <img src="img/TM_NYT_articles.png" height="2000" width="2000">  -->
<!-- </center> -->

### Example: A corpus of food magazines 

![](img/TM_food_magazines.png)

<!-- <center> -->
<!-- <img src="img/TM_food_magazines.png" height="2000" width="2000">  -->
<!-- </center> -->

### A corpus of scientific articles

![](img/TM_science_articles.png)

<!-- <center> -->
<!-- <img src="img/TM_science_articles.png" height="2000" width="2000">  -->
<!-- </center> -->

(Credit: [Dave Blei's presentation](http://www.cs.columbia.edu/~blei/talks/Blei_Science_2008.pdf))

### Topic modeling motivation

- Humans are pretty good at reading and understanding a document and answering questions such as 
    - What is it about?  
    - Which documents is it related to?     
- But for a large collection of documents it would take years to read all documents and organize and categorize them so that they are easy to search.
- You need an automated way
    - to get an idea of what's going on in the data or 
    - to pull documents related to a certain topic

### Topic modeling

- Topic modeling gives you an ability to summarize the major themes in a large collection of documents (corpus). 
    - Example: The major themes in a collection of news articles could be 
        - **politics**
        - **entertainment**
        - **sports**
        - **technology**
        - ...
- A common tool to solve such problems is unsupervised ML methods.
- Given the hyperparameter $K$, the idea of topic modeling is to describe the data using $K$ "topics"

### Topic modeling: Input and output

- Input
    - A large collection of documents
    - A value for the hyperparameter $K$ (e.g., $K = 3$)
- Output
    1. Topic-words association 
        - For each topic, what words describe that topic? 
    2. Document-topics association
        - For each document, what topics are expressed by the document? 
    

### Topic modeling: Example

- Topic-words association 
    - For each topic, what words describe that topic?  
    - A topic is a mixture of words. 

![](img/topic_modeling_word_topics.png)

<!-- <center> -->
<!-- <img src="img/topic_modeling_word_topics.png" height="1000" width="1000">  -->
<!-- </center> -->    

### Topic modeling: Example

- Document-topics association 
    - For each document, what topics are expressed by the document?
    - A document is a mixture of topics. 
    
![](img/topic_modeling_doc_topics.png)

<!-- <center> -->    
<!-- <img src="img/topic_modeling_doc_topics.png" height="800" width="800">  -->
<!-- </center> -->    

### Topic modeling: Input and output

- Input
    - A large collection of documents
    - A value for the hyperparameter $K$ (e.g., $K = 3$)
- Output
    - For each topic, what words describe that topic?  
    - For each document, what topics are expressed by the document?

![](img/topic_modeling_output.png)

<!-- <center> -->
<!-- <img src="img/topic_modeling_output.png" height="800" width="800">  -->
<!-- </center> -->    

### Topic modeling: Some applications

- Topic modeling is a great EDA tool to get a sense of what's going on in a large corpus. 
- Some examples
    - If you want to pull documents related to a particular lawsuit. 
    - You want to examine people's sentiment towards a particular candidate and/or political party and so you want to pull tweets or Facebook posts related to election.   

### Topic modeling toy example

In [ ]:
toy_df = pd.read_csv("data/toy_lda_data.csv")
toy_df

In [ ]:
from gensim import corpora, matutils, models

corpus = [doc.split() for doc in toy_df["text"].tolist()]
# Create a vocabulary for the lda model
dictionary = corpora.Dictionary(corpus)
# Convert our corpus into document-term matrix for Lda
doc_term_matrix = [dictionary.doc2bow(doc) for doc in corpus]

In [ ]:
from gensim.models import LdaModel

# Train an lda model
lda = models.LdaModel(
    corpus=doc_term_matrix,
    id2word=dictionary,
    num_topics=3,
    random_state=123,
    passes=10,
)

In [ ]:
### Examine the topics in our LDA model
lda.print_topics(num_words=4)

In [ ]:
### Examine the topic distribution for a document
print("Document: ", corpus[0])
df = pd.DataFrame(lda[doc_term_matrix[0]], columns=["topic id", "probability"])
df.sort_values("probability", ascending=False)

You can also visualize the topics using `pyLDAvis`. 

```
pip install pyLDAvis

```

> Do not install it using `conda`. They have made some changes in the recent version and `conda` build is not available for this version yet. 

In [ ]:
# Visualize the topics
import pyLDAvis

pyLDAvis.enable_notebook()
import pyLDAvis.gensim_models as gensimvis

vis = gensimvis.prepare(lda, doc_term_matrix, dictionary, sort_topics=False)
vis


## Topic modeling pipeline 

- Preprocess your corpus. 
- Train LDA using `Gensim`.
- Interpret your topics.     

### Data

In [ ]:
import wikipedia

queries = [
    "Artificial Intelligence",
    "unsupervised learning",
    "Supreme Court of Canada",
    "Peace, Order, and Good Government",
    "Canadian constitutional law",
    "ice hockey",
]
wiki_dict = {"wiki query": [], "text": []}
for i in range(len(queries)):
    wiki_dict["text"].append(wikipedia.page(queries[i]).content)
    wiki_dict["wiki query"].append(queries[i])

wiki_df = pd.DataFrame(wiki_dict)
wiki_df

### Preprocessing the corpus 

- **Preprocessing is crucial!**
- Tokenization, converting text to lower case
- Removing punctuation and stopwords
- Discarding words with length < threshold or word frequency < threshold        
- Possibly lemmatization: Consider the lemmas instead of inflected forms. 
- Depending upon your application, restrict to specific part of speech;
    * For example, only consider nouns, verbs, and adjectives
    
We'll use [`spaCy`](https://spacy.io/) for preprocessing. 

In [ ]:
import spacy

nlp = spacy.load("en_core_web_md", disable=["parser", "ner"])

In [ ]:
def preprocess(
    doc,
    min_token_len=2,
    irrelevant_pos=["ADV", "PRON", "CCONJ", "PUNCT", "PART", "DET", "ADP", "SPACE"],
):
    """
    Given text, min_token_len, and irrelevant_pos carry out preprocessing of the text
    and return a preprocessed string.

    Parameters
    -------------
    doc : (spaCy doc object)
        the spacy doc object of the text
    min_token_len : (int)
        min_token_length required
    irrelevant_pos : (list)
        a list of irrelevant pos tags

    Returns
    -------------
    (str) the preprocessed text
    """

    clean_text = []

    for token in doc:
        if (
            token.is_stop == False  # Check if it's not a stopword
            and len(token) > min_token_len  # Check if the word meets minimum threshold
            and token.pos_ not in irrelevant_pos
        ):  # Check if the POS is in the acceptable POS tags
            lemma = token.lemma_  # Take the lemma of the word
            clean_text.append(lemma.lower())
    return " ".join(clean_text)

In [ ]:
wiki_df["text_pp"] = [preprocess(text) for text in nlp.pipe(wiki_df["text"])]

In [ ]:
wiki_df

### Training LDA with [gensim](https://radimrehurek.com/gensim/models/ldamodel.html)

To train an LDA model with [gensim](https://radimrehurek.com/gensim/models/ldamodel.html), you need

- Document-term matrix 
- Dictionary (vocabulary)
- The number of topics ($K$): `num_topics`
- The number of passes: `passes`

### `Gensim`'s `doc2bow`
- Let's first create a dictionary using [`corpora.Dictionary`](https://radimrehurek.com/gensim/corpora/dictionary.html). 

In [ ]:
corpus = [doc.split() for doc in wiki_df["text_pp"].tolist()]
dictionary = corpora.Dictionary(corpus)  # Create a vocabulary for the lda model
pd.DataFrame(
    dictionary.token2id.keys(), index=dictionary.token2id.values(), columns=["Word"]
)

### `Gensim`'s `doc2bow`
- Now let's convert our corpus into document-term matrix for LDA using [`dictionary.doc2bow`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.doc2bow).
- For each document, it stores the frequency of each token in the document in the format (token_id, frequency). 

In [ ]:
doc_term_matrix = [dictionary.doc2bow(doc) for doc in corpus]
doc_term_matrix[1][:20]

Now we are ready to train an LDA model. 

In [ ]:
from gensim.models import LdaModel

num_topics = 3

lda = models.LdaModel(
    corpus=doc_term_matrix,
    id2word=dictionary,
    num_topics=num_topics,
    random_state=42,
    passes=10,
)

### Examine the topics and topic distribution for a document in our LDA model

In [ ]:
lda.print_topics(num_words=4)  # Topics

In [ ]:
print("Document: ", wiki_df.iloc[0][0])
print("Topic assignment for document: ", lda[doc_term_matrix[0]])  # Topic distribution

### Visualize topics

In [ ]:
vis = gensimvis.prepare(lda, doc_term_matrix, dictionary, sort_topics=False)
vis

### (Optional) Topic modeling with `sklearn`

- We are using `Gensim` LDA so that we'll be able to use `CoherenceModel` to evaluate topic model later. 
- Bit we can also train an LDA model with `sklearn`.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()
X = vec.fit_transform(wiki_df["text_pp"])

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

n_topics = 3
lda = LatentDirichletAllocation(
    n_components=n_topics, learning_method="batch", max_iter=10, random_state=0
)
document_topics = lda.fit_transform(X)

In [ ]:
print("lda.components_.shape: {}".format(lda.components_.shape))

In [ ]:
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vec.get_feature_names())

In [ ]:
import mglearn

mglearn.tools.print_topics(
    topics=range(3),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=5,
    n_words=10,
)

<br><br><br><br>

## Basic text preprocessing

### Introduction 
- Why do we need preprocessing?
    - Text data is unstructured and messy. 
    - We need to "normalize" it before we do anything interesting with it. 
- Example:     
    - **Lemma**: Same stem, same part-of-speech, roughly the same meaning
        - Vancouver's &rarr; Vancouver
        - computers &rarr; computer 
        - rising &rarr; rise, rose, rises    

### Tokenization

- Sentence segmentation
    - Split text into sentences
- Word tokenization 
    - Split sentences into words

### Tokenization: sentence segmentation

<blockquote>
MDS is a Master's program at UBC in British Columbia. MDS teaching team is truly multicultural!! Dr. George did his Ph.D. in Scotland. Dr. Timbers, Dr. Ostblom, Dr. Rodríguez-Arelis, and Dr. Kolhatkar did theirs in Canada. Dr. Gelbart did his PhD in the U.S.
</blockquote>

- How many sentences are there in this text? 

In [ ]:
### Let's do sentence segmentation on "."
text = (
    "MDS is a Master's program at UBC in British Columbia. "
    "MDS teaching team is truly multicultural!! "
    "Dr. George did his Ph.D. in Scotland. "
    "Dr. Timbers, Dr. Ostblom, Dr. Rodríguez-Arelis, and Dr. Kolhatkar did theirs in Canada. "
    "Dr. Gelbart did his PhD in the U.S."
)

print(text.split("."))

### Sentence segmentation

- In English, period (.) is quite ambiguous. (In Chinese, it is unambiguous.)
    - Abbreviations like Dr., U.S., Inc.  
    - Numbers like 60.44%, 0.98
- ! and ? are relatively ambiguous.
- How about writing regular expressions? 
- A common way is using off-the-shelf models for sentence segmentation. 

In [ ]:
### Let's try to do sentence segmentation using nltk
from nltk.tokenize import sent_tokenize

sent_tokenized = sent_tokenize(text)
print(sent_tokenized)

### Word tokenization

<blockquote>
MDS is a Master's program at UBC in British Columbia. 
</blockquote>

- How many words are there in this sentence?  
- Is whitespace a sufficient condition for a word boundary?

### Word tokenization 

<blockquote>
MDS is a Master's program at UBC in British Columbia. 
</blockquote>

- What's our definition of a word?
    - Should British Columbia be one word or two words? 
    - Should punctuation be considered a separate word?
    - What about the punctuations in `U.S.`?
    - What do we do with words like `Master's`?
- This process of identifying word boundaries is referred to as **tokenization**.
- You can use regex but better to do it with off-the-shelf ML models.  

In [ ]:
### Let's do word segmentation on white spaces
print("Splitting on whitespace: ", [sent.split() for sent in sent_tokenized])

### Let's try to do word segmentation using nltk
from nltk.tokenize import word_tokenize

word_tokenized = [word_tokenize(sent) for sent in sent_tokenized]
# This is similar to the input format of word2vec algorithm
print("\n\n\nTokenized: ", word_tokenized)

### Word segmentation 

For some languages you need much more sophisticated tokenizers. 
- For languages such as Chinese, there are no spaces between words.
    - [jieba](https://github.com/fxsjy/jieba) is a popular tokenizer for Chinese. 
- German doesn't separate compound words.
    * Example: _rindfleischetikettierungsüberwachungsaufgabenübertragungsgesetz_
    * (the law for the delegation of monitoring beef labeling)

### Types and tokens
- Usually in NLP, we talk about 
    - **Type** an element in the vocabulary
    - **Token** an instance of that type in running text 


### Exercise for you 

<blockquote>    
UBC is located in the beautiful province of British Columbia. It's very close 
to the U.S. border. You'll get to the USA border in about 45 mins by car.     
</blockquote>  

- Consider the example above. 
    - How many types? (task dependent)
    - How many tokens? 

### Other commonly used preprocessing steps

- Punctuation and stopword removal
- Stemming and lemmatization

### Punctuation and stopword removal

- The most frequently occurring words in English are not very useful in many NLP tasks.
    - Example: _the_ , _is_ , _a_ , and punctuation
- Probably not very informative in many tasks 

In [ ]:
# Let's use `nltk.stopwords`.
# Add punctuations to the list.
stop_words = list(set(stopwords.words("english")))
import string

punctuation = string.punctuation
stop_words += list(punctuation)
# stop_words.extend(['``','`','br','"',"”", "''", "'s"])
print(stop_words)

In [ ]:
### Get rid of stop words
preprocessed = []
for sent in word_tokenized:
    for token in sent:
        token = token.lower()
        if token not in stop_words:
            preprocessed.append(token)
print(preprocessed)

### Lemmatization 

- For many NLP tasks (e.g., web search) we want to ignore morphological differences between words
    - Example: If your search term is "studying for ML quiz" you might want to include pages containing "tips to study for an ML quiz" or "here is how I studied for my ML quiz"
- Lemmatization converts inflected forms into the base form. 

In [ ]:
import nltk

nltk.download("wordnet")

In [ ]:
# nltk has a lemmatizer
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
print("Lemma of studying: ", lemmatizer.lemmatize("studying", "v"))
print("Lemma of studied: ", lemmatizer.lemmatize("studied", "v"))

### Stemming

- Has a similar purpose but it is a crude chopping of affixes 
    * _automates, automatic, automation_ all reduced to _automat_.
- Usually these reduced forms (stems) are not actual words themselves.  
- A popular stemming algorithm for English is PorterStemmer. 
- Beware that it can be aggressive sometimes.

In [ ]:
from nltk.stem.porter import PorterStemmer

text = (
    "UBC is located in the beautiful province of British Columbia... "
    "It's very close to the U.S. border."
)
ps = PorterStemmer()
tokenized = word_tokenize(text)
stemmed = [ps.stem(token) for token in tokenized]
print("Before stemming: ", text)
print("\n\nAfter stemming: ", " ".join(stemmed))

### Other tools for preprocessing 

- We used [Natural Language Processing Toolkit (nltk)](https://www.nltk.org/) above
- Many available tools    
- [spaCy](https://spacy.io/)

### [spaCy](https://spacy.io/)

- Industrial strength NLP library. 
- Lightweight, fast, and convenient to use. 
- spaCy does many things that we did above in one line of code! 
- Also has [multi-lingual](https://spacy.io/models/xx) support. 

In [ ]:
import spacy

# Load the model
nlp = spacy.load("en_core_web_md")
text = (
    "MDS is a Master's program at UBC in British Columbia. "
    "MDS teaching team is truly multicultural!! "
    "Dr. George did his Ph.D. in Scotland. "
    "Dr. Timbers, Dr. Ostblom, Dr. Rodríguez-Arelis, and Dr. Kolhatkar did theirs in Canada. "
    "Dr. Gelbart did his PhD in the U.S."
)

doc = nlp(text)

In [ ]:
# Accessing tokens
tokens = [token for token in doc]
print("\nTokens: ", tokens)

# Accessing lemma
lemmas = [token.lemma_ for token in doc]
print("\nLemmas: ", lemmas)

# Accessing pos
pos = [token.pos_ for token in doc]
print("\nPOS: ", pos)

### Other typical NLP tasks 
In order to understand text, we usually are interested in extracting information from text. Some common tasks in NLP pipeline are: 
- Part of speech tagging
    - Assigning part-of-speech tags to all words in a sentence.
- Named entity recognition
    - Labelling named “real-world” objects, like persons, companies or locations.    
- Coreference resolution
    - Deciding whether two strings (e.g., UBC vs University of British Columbia) refer to the same entity
- Dependency parsing
    - Representing grammatical structure of a sentence

### Extracting named-entities using spaCy

In [ ]:
from spacy import displacy

doc = nlp(
    "University of British Columbia "
    "is located in the beautiful "
    "province of British Columbia."
)
displacy.render(doc, style="ent")
# Text and label of named entity span
print("Named entities:\n", [(ent.text, ent.label_) for ent in doc.ents])
print("\nORG means: ", spacy.explain("ORG"))
print("GPE means: ", spacy.explain("GPE"))

### Dependency parsing using spaCy

In [ ]:
doc = nlp("I like cats")
displacy.render(doc, style="dep")

### Many other things possible

- A powerful tool 
- All my Capstone groups last year used this tool. 
- You can build your own rule-based searches. 
- You can also access word vectors using spaCy with bigger models. (Currently we are using `en_core_web_md` model.)

<br><br><br><br>